In [56]:
import math
import csv
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
import numpy.linalg as lag

csv_arr = []

#The csv reader
with open('cp2_input.csv') as csv_in:
    filereader = csv.reader(csv_in)
    for row in filereader:
        csv_arr.append(row)

two_sig_a = [float(i) for i in csv_arr[0]]
#The fission matrix rows were precalculated
two_fiss_row1 = [float(i) for i in csv_arr[1]]
two_fiss_row2 = [float(i) for i in csv_arr[2]]
two_in_row1 = [float(i) for i in csv_arr[3]]
two_in_row2 = [float(i) for i in csv_arr[4]]
two_out = [float(i) for i in csv_arr[5]]
eight_sig_a = [float(i) for i in csv_arr[6]]
eight_fiss_row1 = [float(i) for i in csv_arr[7]]
eight_fiss_row2 = [float(i) for i in csv_arr[8]]
eight_fiss_row3 = [float(i) for i in csv_arr[9]]
eight_in_row1 = [float(i) for i in csv_arr[10]]
eight_in_row2 = [float(i) for i in csv_arr[11]]
eight_in_row3 = [float(i) for i in csv_arr[12]]
eight_in_row4 = [float(i) for i in csv_arr[13]]
eight_in_row5 = [float(i) for i in csv_arr[14]]
eight_in_row6 = [float(i) for i in csv_arr[15]]
eight_in_row7 = [float(i) for i in csv_arr[16]]
eight_in_row8 = [float(i) for i in csv_arr[17]]
eight_out = [float(i) for i in csv_arr[18]]
#Xi values at group 4 onward were 0, so those values are zero too
zero = list(np.zeros(8))

#This takes outscattering values and makes an ordered array of them
def outscattering(array):
    outscatter = np.zeros((len(array),len(array)))
    for i in range(len(array)):
        outscatter[i][i] = array[i]
    return outscatter

#Defining the matrices
two_outscatter = outscattering(two_out)
two_absorption = outscattering(two_sig_a)
two_inscatter = [two_in_row1,two_in_row2]
two_fission = [two_fiss_row1,two_fiss_row2]
eight_absorption = outscattering(eight_sig_a)
eight_fission = [eight_fiss_row1,eight_fiss_row2,eight_fiss_row3,zero,zero,zero,zero,zero]
eight_inscatter = [eight_in_row1,eight_in_row2,eight_in_row3,eight_in_row4,eight_in_row5,eight_in_row6,eight_in_row7,eight_in_row8]
eight_outscatter = outscattering(eight_out)
#These are all the calculated matrices
two_migration = np.subtract(np.add(two_absorption,two_outscatter),two_inscatter)
two_B = np.dot(lag.inv(two_migration),two_fission)
#Eigenvalue and vector of the two group
#I keep the eigenvectors in the form numpy gives them, as to not clutter the code with for loops parsing it
two_eigenval,two_eigenvec = lag.eig(two_B)
eight_migration = np.subtract(np.add(eight_absorption,eight_outscatter),eight_inscatter)
eight_B = np.dot(lag.inv(eight_migration),eight_fission)
#Eigenvalue and vector of the eight group
eight_eigenval,eight_eigenvec = lag.eig(eight_B)


print(two_eigenval)
print(two_eigenvec)
print(eight_eigenval)
print(eight_eigenvec)

[1.11022302e-16 9.96138799e-01]
[[-0.99918547 -0.97730867]
 [ 0.04035341 -0.21182012]]
[ 1.09354393e+00+0.00000000e+00j  2.25573958e-03+0.00000000e+00j
  6.77558387e-06+0.00000000e+00j  2.39370866e-17+0.00000000e+00j
  7.72272390e-18+7.69082255e-18j  7.72272390e-18-7.69082255e-18j
 -3.46567307e-18+1.33951188e-18j -3.46567307e-18-1.33951188e-18j]
[[-2.60717427e-01+0.00000000e+00j -7.62162536e-01+0.00000000e+00j
  -9.83778469e-01+0.00000000e+00j -2.79809294e-01+0.00000000e+00j
  -1.97582297e-01-1.04302948e-01j -1.97582297e-01+1.04302948e-01j
  -6.10243500e-02+2.09023573e-01j -6.10243500e-02-2.09023573e-01j]
 [-3.44801168e-01+0.00000000e+00j  6.42292206e-01+0.00000000e+00j
   1.51966249e-03+0.00000000e+00j -1.38815186e-14+0.00000000e+00j
  -3.63900073e-15-1.83351936e-15j -3.63900073e-15+1.83351936e-15j
  -1.56325559e-16+1.99176776e-15j -1.56325559e-16-1.99176776e-15j]
 [-7.81972156e-01+0.00000000e+00j  6.95540916e-02+0.00000000e+00j
   1.55987964e-01+0.00000000e+00j  6.27975775e-01+0.0000

In [48]:
#Power iteration
def power_iteration(T,num_sim):
    phi_k = np.ones(len(T[0]))
    k = np.ones(len(T[0]))
    for i in range(num_sim):
        phi_k1 = np.dot(T,phi_k)
        phi_k1_norm = lag.norm(phi_k1)
        phi_k = phi_k1 / phi_k1_norm
        this = np.dot(np.transpose(np.dot(T,phi_k)),phi_k)
        that = np.dot(np.transpose(phi_k),phi_k)
        k = this/that
    return phi_k,k
two_pow_phi,two_powk = power_iteration(two_B,2)
eight_pow_phi,eight_powk = power_iteration(eight_B,2)
print(power_iteration(two_B,2))
print(power_iteration(eight_B,2))

(array([0.97730867, 0.21182012]), 0.9961387988671836)
(array([0.26071822, 0.34480055, 0.78197216, 0.28035377, 0.22326264,
       0.11761087, 0.10450718, 0.22013545]), 1.0935440381142285)


In [54]:
#Writing to an output file
with open('cp2_output.csv', mode='w') as output_file:
    filewriter = csv.writer(output_file, delimiter=',', lineterminator='\n', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(two_sig_a)
    filewriter.writerow(two_fiss_row1)
    filewriter.writerow(two_fiss_row2)
    filewriter.writerow(two_in_row1)
    filewriter.writerow(two_in_row2)
    filewriter.writerow(two_out)
    filewriter.writerow(eight_sig_a)
    filewriter.writerow(eight_fiss_row1)
    filewriter.writerow(eight_fiss_row2)
    filewriter.writerow(eight_fiss_row3)
    filewriter.writerow(eight_in_row1)
    filewriter.writerow(eight_in_row2)
    filewriter.writerow(eight_in_row3)
    filewriter.writerow(eight_in_row4)
    filewriter.writerow(eight_in_row5)
    filewriter.writerow(eight_in_row6)
    filewriter.writerow(eight_in_row7)
    filewriter.writerow(eight_in_row8)
    filewriter.writerow(eight_out)
    filewriter.writerow(two_migration)
    filewriter.writerow(two_fission)
    filewriter.writerow(two_eigenval)
    filewriter.writerow(two_eigenvec)
    filewriter.writerow(two_pow_phi)
    filewriter.writerow([two_powk])
    filewriter.writerow(eight_migration)
    filewriter.writerow(eight_fission)
    filewriter.writerow(eight_eigenval)
    filewriter.writerow(eight_eigenvec)
    filewriter.writerow(eight_pow_phi)
    filewriter.writerow([eight_powk])